## Importa funções para o pré-processamento

In [1]:
from pre_processamento import *

from dlisio import dlis
import pandas as pd

# A função 'glob' do módulo 'glob' é usada para procurar todos os arquivos em um diretório com determinada extensão
import glob

## Carrega os dados

In [2]:
dli_dict = {}        # Conterá os arquivos lógicos
filenames = []  # Conterá os caminhos dos arquivos dlis
nomes_anp = []      # Conterá os nomes ANP dos poços
charId_inicio, charId_fim = 7, 10   # Índices de início e fim da substring identificadora do poço

# Utilizando a função 'glob' do módulo 'glob' para procurar todos os arquivos com extensão DLIS em 'Data'
for file in glob.glob(r'**/Data' + "/*.dlis", recursive=True):
    
    # Carregando os arquivos dlis com 'load' e armazenando-os em 'leitura'
    # 'tail' recebe valores restantes, caso a função retorne mais de uma peça de informação
    leitura, *tail = dlis.load(f'{file}')
    
    nome = leitura.origins[0].well_name

    nomes_anp.append(nome)    # armazenando o nome do poço
    
    filenames.append(file)      # armazenando o caminho dos arquivos dlis
    
    nome_abreviado = nome[charId_inicio : charId_fim]   # identificador do poço, e.g. '900'
    
    # Armazenando o arquivo lógico como valor em um dicionário onde a chave é o identificador do poço
    dli_dict[nome_abreviado] = leitura   

## Separa TODAS as curvas presentes nos .dlis de cada poço

In [14]:
channels_dict = {}

for key, poco in dli_dict.items():
    channels = poco.frames[0].channels
    channels_list = [channel.name for channel in channels]
    channels_dict[key] = channels_list

## Salva as curvas presentes nos .dlis em arquivos CSV

In [80]:
import csv

for key, value in channels_dict.items():
    file_name = f"curvas_{key}.csv"
    with open(file_name, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow([f"Curvas do Poco {key}"])
        for item in value:
            writer.writerow([item])

    print(f"Arquivo {file_name} criado com sucesso")

Arquivo curvas_900.csv criado com sucesso
Arquivo curvas_889.csv criado com sucesso
Arquivo curvas_912.csv criado com sucesso
Arquivo curvas_897.csv criado com sucesso
Arquivo curvas_910.csv criado com sucesso
Arquivo curvas_907.csv criado com sucesso


## Cria dataframes para os poços

In [82]:
dlis_df_dict = {}   # Conterá os dataframes respectivos aos poços

# Curvas de perfis escolhidas
curvas_escolhidas = ['TDEP', 'GR', 'NPHI', 'RHOB', 'DRHO', 'HDRS', 'LLD', 'BSZ', 'BS', 'CALI', 'DCALI', 'PE', 'DTC']

# Iterando sobre os arquivos lógicos de todos os poços (que estão armazenados em 'dli_dict'),
# '.values()' se refere aos valores do dicionário (não às chaves)
for chave, poco in dli_dict.items():
    try:
        # Armazenando as curvas que serão utilizadas em uma lista
        curvas_utilizadas = [
            channel.name                                    # Os elementos da lista serão os nomes das curvas
            for channel in poco.channels                    # As curvas são acessadas por meio de 'poco.channels'
            if channel.name in curvas_escolhidas            # As curvas que não utilizaremos não serão armazenadas na lista
        ]


        curvas = poco.frames[0].curves()

        # Criando um pandas dataframe do poço respectivo à atual iteração e armazenando o mesmo em dlis_df
        dlis_df_dict[chave] = pd.DataFrame(curvas[curvas_utilizadas])
    except:
        pass

## Aplicando os mnemônicos

In [83]:
aplicaMnemonico(dlis_df_dict, 'BS', 'BSZ')
aplicaMnemonico(dlis_df_dict, 'LLD', 'HDRS')

In [86]:
for key, poco in dlis_df_dict.items():
    curvas = sorted(poco.keys())
    print(f"{key}: {curvas}")

900: ['BSZ', 'CALI', 'DRHO', 'DTC', 'GR', 'HDRS', 'NPHI', 'PE', 'RHOB', 'TDEP']
889: ['BSZ', 'CALI', 'DRHO', 'DTC', 'GR', 'HDRS', 'NPHI', 'PE', 'RHOB', 'TDEP']
912: ['BSZ', 'CALI', 'DRHO', 'DTC', 'GR', 'HDRS', 'NPHI', 'PE', 'RHOB', 'TDEP']
897: ['BSZ', 'CALI', 'DRHO', 'DTC', 'GR', 'HDRS', 'NPHI', 'PE', 'RHOB', 'TDEP']
910: ['BSZ', 'CALI', 'DRHO', 'DTC', 'GR', 'HDRS', 'NPHI', 'PE', 'RHOB', 'TDEP']
907: ['BSZ', 'CALI', 'DRHO', 'DTC', 'GR', 'HDRS', 'NPHI', 'PE', 'RHOB', 'TDEP']


In [87]:
# Transformados os valores -999.25 em nulos
for poco in dlis_df_dict.values():
    poco.replace([-999.25], [None], inplace = True)

## Adiciona coluna DCALI

In [88]:
add_DCALI(dlis_df_dict)

limita_curva(dlis_df_dict, "DCALI", -2, 2)

## Remove valores DRHO indesejados

In [89]:
limita_curva(dlis_df_dict, "DRHO", -0.15, 0.15)

## Inverte a ordem das linhas dos dataframes

In [98]:
for key in dlis_df_dict.keys():
    dlis_df_dict[key] = dlis_df_dict[key].iloc[::-1]

## Salva os dados dos dataframes em arquivos CSV

In [100]:
for key, value in dlis_df_dict.items():
    file_name = f"Data_CSV/poco_{key}.csv"
    value.to_csv(file_name, index=False)
    print(f"Arquivo {file_name} criado com sucesso.")

Arquivo Data_CSV/poco_900.csv criado com sucesso.
Arquivo Data_CSV/poco_889.csv criado com sucesso.
Arquivo Data_CSV/poco_912.csv criado com sucesso.
Arquivo Data_CSV/poco_897.csv criado com sucesso.
Arquivo Data_CSV/poco_910.csv criado com sucesso.
Arquivo Data_CSV/poco_907.csv criado com sucesso.


In [99]:
pd.set_option('display.max_rows', None)
dlis_df_dict['900']

,TDEP,GR,BSZ,NPHI,CALI,RHOB,DRHO,PE,HDRS,DTC,DCALI
3599,14.999989,45.702099,8.5,None,None,None,None,None,None,None,NaN
3598,15.099989,45.021198,8.5,None,None,None,None,None,None,None,NaN
3597,15.199989,48.504501,8.5,None,None,None,None,None,None,None,NaN
3596,15.299989,52.5182,8.5,None,None,None,None,None,None,None,NaN
3595,15.399989,55.795601,8.5,None,None,None,None,None,None,None,NaN
3594,15.499989,58.6693,8.5,None,None,None,None,None,None,None,NaN
3593,15.599989,58.0518,8.5,None,None,None,None,None,None,None,NaN
3592,15.699989,57.893501,8.5,None,None,None,None,None,None,None,NaN
3591,15.799989,54.624001,8.5,None,None,None,None,None,None,None,NaN
3590,15.899989,52.755699,8.5,None,None,None,None,None,None,None,NaN


In [15]:
dlis_df_dict['900']

,TDEP,GR,BSZ,NPHI,CALI,RHOB,DRHO,PE,HDRS,DTC,DCALI
0,374.899994,None,8.5,None,None,None,None,None,None,None,NaN
1,374.799994,None,8.5,None,None,None,None,None,None,None,NaN
2,374.699994,None,8.5,None,None,None,None,None,None,None,NaN
3,374.599994,None,8.5,None,None,None,None,None,None,None,NaN
4,374.499994,None,8.5,None,None,None,None,None,None,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...
3595,15.399989,55.795601,8.5,None,None,None,None,None,None,None,NaN
3596,15.299989,52.5182,8.5,None,None,None,None,None,None,None,NaN
3597,15.199989,48.504501,8.5,None,None,None,None,None,None,None,NaN
3598,15.099989,45.021198,8.5,None,None,None,None,None,None,None,NaN
